In [9]:
! pip install -r requirements.txt

In [8]:
PROJECT_ID = "fantasymaps-334622"
BUCKET = "fantasy-maps"
LOCATION = "us-central1"
MODEL_ID = "4304645197347684352"

In [ ]:
# Test batch prediction op
from google.cloud import aiplatform as aip
aip.init(project=PROJECT_ID, location=LOCATION)

input_file = "gs://fantasy-maps/ScrapedData/_batch_prediction_input_20221216191812.jsonl"
model = aip.Model(f"projects/{PROJECT_ID}/locations/{LOCATION}/models/{MODEL_ID}")

batch_prediction_job = model.batch_predict(
        job_display_name="test-batch-predict-job",
        gcs_source=[input_file],
        gcs_destination_prefix="gs://fantasy-maps/ScrapedData/output",
        sync=True,
    )

batch_prediction_job.wait()

print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/733537716875/locations/us-central1/batchPredictionJobs/4435987496282619904
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/733537716875/locations/us-central1/batchPredictionJobs/4435987496282619904')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/4435987496282619904?project=733537716875
BatchPredictionJob projects/733537716875/locations/us-central1/batchPredictionJobs/4435987496282619904 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/733537716875/locations/us-central1/batchPredictionJobs/4435987496282619904 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/733537716875/locations/us-central1/batchPredictionJobs/4435987496282619904 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/733537716875/locations/us-central1/batchPredictionJobs/

In [10]:
#help(batch_prediction_job)
print(batch_prediction_job.output_info.gcs_output_directory)
print(type(batch_prediction_job.output_info))

gs://fantasy-maps/ScrapedData/output/prediction-dnd-maps-full-grid-model-online-20220803020036-2023-01-26T19:59:52.442336Z
<class 'google.cloud.aiplatform_v1.types.batch_prediction_job.BatchPredictionJob.OutputInfo'>


In [11]:
from google.cloud import storage

s_client = storage.Client(project=PROJECT_ID)
bucket = s_client.bucket(BUCKET)

blobs = bucket.list_blobs(prefix="ScrapedData")
for b in blobs:
    if b.name.find("jsonl") > -1:
        print(b.name)

ScrapedData/_batch_prediction_input_20221216182618.jsonl
ScrapedData/_batch_prediction_input_20221216191812.jsonl
ScrapedData/_batch_prediction_input_20221220211236.jsonl
ScrapedData/_batch_prediction_input_20230109001303.jsonl
ScrapedData/_batch_prediction_input_20230109010101.jsonl
ScrapedData/_batch_prediction_input_20230122213956.jsonl
ScrapedData/_batch_prediction_input_20230123003048.jsonl
ScrapedData/bp_input_20220405213332.jsonl
ScrapedData/bp_input_20220405214819.jsonl
ScrapedData/bp_input_20220405232932.jsonl
ScrapedData/bp_input_20220405232937.jsonl
ScrapedData/bp_input_20220413022208.jsonl
ScrapedData/bp_input_20220413022217.jsonl
ScrapedData/bp_input_20220413023017.jsonl
ScrapedData/bp_input_20220413023033.jsonl
ScrapedData/bp_input_20220508022059.jsonl
ScrapedData/bp_input_20220508023145.jsonl
ScrapedData/bp_input_20220508031351.jsonl
ScrapedData/bp_input_20220508031811.jsonl
ScrapedData/bp_input_20220508181800.jsonl
ScrapedData/bp_input_20220508185029.jsonl
ScrapedData/b

In [12]:
import jsonlines

uri = "ScrapedData/output/prediction-dnd-maps-full-grid-model-online-20220803020036-2023-01-26T19:59:52.442336Z/predictions_00001.jsonl"

b = bucket.blob(uri)
bp_jsonl = b.download_as_string()

predictions = bp_jsonl.decode("utf-8").split("\n")
reader = jsonlines.Reader(predictions)

images_uris = {
    "usable": [],
    "unusable": [],
}

for obj in reader.iter(type=dict, skip_invalid=True):
    confidences = obj["prediction"]["confidences"]
    image_gcs_uri = obj["instance"]["content"]
    # NOTE: we check the predictions above the percentage of
    # to ensure they are above the acceptable threshold. If
    # the image is above the acceptable threshold, we keep
    # the image for training data.
    top_n_images = int(len(confidences) * 0.1)
    marginal_result = confidences[top_n_images]
    if marginal_result > 0.3:
        images_uris["usable"].append(image_gcs_uri)
    else:
        images_uris["unusable"].append(image_gcs_uri)

print(images_uris)

{'usable': ['gs://fantasy-maps/ScrapedData/abandoned_iron_mine_entrance.25x35.jpg', 'gs://fantasy-maps/ScrapedData/underground_fighting_pit.22x29.jpg', 'gs://fantasy-maps/ScrapedData/ramshackle_tavern.15x15.jpg', 'gs://fantasy-maps/ScrapedData/small_camp_side_forest_road_campers.20x40.jpg', 'gs://fantasy-maps/ScrapedData/path_snowy_hills.25x30.jpg', 'gs://fantasy-maps/ScrapedData/arrival_vyrlich.20x20.jpg', 'gs://fantasy-maps/ScrapedData/free_day_night_maps_frozen_lakeside.42x36.jpg', 'gs://fantasy-maps/ScrapedData/processing.30x30.jpg', 'gs://fantasy-maps/ScrapedData/virlych_hungry_moutains_top_evil_gate.40x30.jpg', 'gs://fantasy-maps/ScrapedData/free_alien_biome_battlemap.30x30.jpg', 'gs://fantasy-maps/ScrapedData/christmas_battle_map.40x30.jpg', 'gs://fantasy-maps/ScrapedData/archaeologist_camp.50x50.jpg', 'gs://fantasy-maps/ScrapedData/hungry_mountain_summit_evil_gate_virlych.40x30.jpg', 'gs://fantasy-maps/ScrapedData/arrival_vyrlich_roots_evil.30x20.jpg', 'gs://fantasy-maps/Scrape

In [13]:
from google.cloud import storage
from PIL import Image
from io import BytesIO

img_uri = "ScrapedData/15x60_illmarsh_tunnels_under_t.15x60.jpg"

s_client = storage.Client(PROJECT_ID)
bucket = s_client.bucket(BUCKET)

blob = bucket.blob(img_uri)
img_bytes = blob.download_as_bytes()

w = h = 0
f = BytesIO(img_bytes)
with Image.open(f) as img:
    w, h = img.size

In [14]:
print(f"{w}, {h}")

2048, 512


In [15]:
def process_batch_predict_output(bp_jsonl, threshold, percentage):
    """Parses a set of batch predictions for highest confidence inputs.

    Arguments:
        bp_jsonl (str): the batch prediction results, as a string
        threshold (float): the lowest confidence value to accept from 0.0 to 1.0
        percentage (float): the top percentage (by quality) of predictions to check

    Returns:
        list of GCS URIs
    """
    import jsonlines

    predictions = bp_jsonl.decode("utf-8").split("\n")
    reader = jsonlines.Reader(predictions)

    images_uris = {
        "usable": [],
        "unusable": [],
    }

    for obj in reader.iter(type=dict, skip_invalid=True):
        confidences = obj["prediction"]["confidences"]
        image_gcs_uri = obj["instance"]["content"]
        # NOTE: we check the predictions above the percentage of
        # to ensure they are above the acceptable threshold. If
        # the image is above the acceptable threshold, we keep
        # the image for training data.
        top_n_images = int(len(confidences) * percentage)
        marginal_result = confidences[top_n_images]
        if marginal_result > threshold:
            images_uris["usable"].append(image_gcs_uri)
        else:
            images_uris["unusable"].append(image_gcs_uri)

    return images_uris

In [16]:
batch_prediction_uri = uri
bp_json_blob = ""

In [23]:
from google.cloud import storage
from google.cloud import firestore

from io import BytesIO
import pandas as pd

project_id = PROJECT_ID
gcs_bucket_name = BUCKET
collection_name = "FantasyMapsTest"
csv_input_file = "ScrapedData/_reddit-scraped-battlemaps-20230123002739.csv"

threshold = 0.3
percentage = 0.1

storage_client = storage.Client(project=project_id)
bucket = storage_client.bucket(gcs_bucket_name)

firestore_client = firestore.Client(project=project_id)
collection_ref = firestore_client.collection(collection_name)

# Determine how many of the scraped data is usable for training a model
bp_path = batch_prediction_uri.replace(f"gs://{gcs_bucket_name}/", "")

blobs = bucket.list_blobs(prefix=bp_path)
for b in blobs:
    if b.name.find(".jsonl") > -1:
        bp_jsonl_blob = b
        break

bp_jsonl = bp_jsonl_blob.download_as_string()

image_uris = process_batch_predict_output(bp_jsonl, threshold, percentage)

# Open up the complete list of scraped images as a DataFrame
blob = bucket.blob(csv_input_file)
csv_bytes = blob.download_as_string()
csv_buffer = BytesIO(csv_bytes)

jpg_df = pd.read_csv(csv_buffer)
shards_df = pd.DataFrame()

print(jpg_df.head)

# Iterate over JPG URIs, download them in batches, convert to sha values
for i, row in jpg_df.iterrows():
    jpg_uri = row["URI"]
    filename = row["Filename"]
    
    print(jpg_uri)

    if jpg_uri in image_uris["unusable"]:
        continue

    if jpg_uri is "nan":
        print(row)
        continue
    
    jpg_uri = jpg_uri.replace(f"gs://{BUCKET}/", "")
    jpg_blob = bucket.blob(jpg_uri)
    jpg_bytes = jpg_blob.download_as_bytes()

<bound method NDFrame.head of     Unnamed: 0  Unnamed: 0.1  \
0            0             4   
1            1             8   
2            2            13   
3            3            17   
4            4            21   
..         ...           ...   
89          89           280   
90          90           283   
91          91           284   
92          92           286   
93          93           294   

                                                Title  Post       ID  \
0             Dwarven Throne Hall - Spider Lord 29x58   NaN  10il9u8   
1              Snowy Mountain Temple Battle Map 30x30   NaN  10i1hly   
2   Infected Cave Battlemap | Catacombs Map | 33x7...   NaN  10ipjde   
3                                 Fallen Tree [27x23]   NaN  10hzfqy   
4                        The Archer's Retreat (37x37)   NaN  10i7yj9   
..                                                ...   ...      ...   
89                            Trapdoor Hideout (8x11)   NaN  10784u3   
90    Dag